# Simple Neural Attentive Meta-Learner (SNAIL)

TODO: Allow GPU

TODO: Fix multiprocessing warning from num_workers in torch

In [1]:
# Ignore InterpolationMode warnings from torchmeta
import warnings
warnings.filterwarnings("ignore")

## Imports

In [2]:
from collections import OrderedDict
import math
import os

import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchmeta.datasets.helpers import omniglot
from torchmeta.utils.data import BatchMetaDataLoader

## Classes

### `blocks.py`

In [3]:
class CasualConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, 
                 stride=1, dilation=1, groups=1, bias=True):
        super(CasualConv1d, self).__init__()
        self.dilation = dilation
        padding = dilation * (kernel_size - 1)
        self.conv1d = nn.Conv1d(in_channels, out_channels, kernel_size, stride,
                                padding, dilation, groups, bias)

    def forward(self, input):
        # Takes something of shape (N, in_channels, T),
        # returns (N, out_channels, T)
        out = self.conv1d(input)
        return out[:, :, :-self.dilation] # TODO: make this correct for different strides/padding

In [4]:
class DenseBlock(nn.Module):
    def __init__(self, in_channels, dilation, filters, kernel_size=2):
        super(DenseBlock, self).__init__()
        self.casualconv1 = CasualConv1d(in_channels, filters, kernel_size, dilation=dilation)
        self.casualconv2 = CasualConv1d(in_channels, filters, kernel_size, dilation=dilation)

    def forward(self, input_):
        # input is dimensions (N, in_channels, T)
        xf = self.casualconv1(input_)
        xg = self.casualconv2(input_)
        activations = torch.tanh(xf) * torch.sigmoid(xg) # shape: (N, filters, T)

        return torch.cat((input_, activations), dim=1)

In [5]:
class TCBlock(nn.Module):
    def __init__(self, in_channels, seq_length, filters):
        super(TCBlock, self).__init__()
        self.dense_blocks = nn.ModuleList([DenseBlock(in_channels + i * filters, 2 ** (i+1), filters)
                                           for i in range(int(math.ceil(math.log(seq_length, 2))))])

    def forward(self, input):
        # input is dimensions (N, T, in_channels)
        input = torch.transpose(input, 1, 2)
        for block in self.dense_blocks:
            input = block(input)
        return torch.transpose(input, 1, 2)

In [6]:
class AttentionBlock(nn.Module):
    def __init__(self, in_channels, key_size, value_size):
        super(AttentionBlock, self).__init__()
        self.linear_query = nn.Linear(in_channels, key_size)
        self.linear_keys = nn.Linear(in_channels, key_size)
        self.linear_values = nn.Linear(in_channels, value_size)
        self.sqrt_key_size = math.sqrt(key_size)

    def forward(self, input):
        # input is dim (N, T, in_channels) where N is the batch_size, and T is
        # the sequence length
        mask = np.array([[1 if i>j else 0 for i in range(input.shape[1])] for j in range(input.shape[1])])
#         mask = torch.BoolTensor(mask).cuda()
        mask = torch.BoolTensor(mask)

        #import pdb; pdb.set_trace()
        keys = self.linear_keys(input) # shape: (N, T, key_size)
        query = self.linear_query(input) # shape: (N, T, key_size)
        values = self.linear_values(input) # shape: (N, T, value_size)
        temp = torch.bmm(query, torch.transpose(keys, 1, 2)) # shape: (N, T, T)
        temp.data.masked_fill_(mask, -float('inf'))
        temp = F.softmax(temp / self.sqrt_key_size, dim=1) # shape: (N, T, T), broadcasting over any slice [:, x, :], each row of the matrix
        temp = torch.bmm(temp, values) # shape: (N, T, value_size)
        return torch.cat((input, temp), dim=2) # shape: (N, T, in_channels + value_size)

### `resnet_blocks.py`

In [7]:
def conv(in_planes, out_planes, kernel_size=3, stride=1, padding=1):
    """convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride,
                     padding=padding, bias=False)

In [8]:
def conv_block(in_channels, out_channels):
    '''
    returns a block conv-bn-relu-pool
    '''
    return nn.Sequential(OrderedDict([
        ('conv', nn.Conv2d(in_channels, out_channels, 3, padding=1)),
        ('bn', nn.BatchNorm2d(out_channels, momentum=1)),
        #('bn', nn.BatchNorm2d(out_channels)),
        ('relu', nn.ReLU()),
        ('pool', nn.MaxPool2d(2))
    ]))

In [9]:
def batchnorm(input, weight=None, bias=None, running_mean=None, running_var=None, training=True,eps=1e-5, momentum=0.1):
    # momentum = 1 restricts stats to the current mini-batch
    # This hack only works when momentum is 1 and avoids needing to track
    # running stats by substituting dummy variables
    size = int(np.prod(np.array(input.data.size()[1])))
#     running_mean = torch.zeros(size).cuda()
#     running_var = torch.ones(size).cuda()
    running_mean = torch.zeros(size)
    running_var = torch.ones(size)
    return F.batch_norm(input, running_mean, running_var, weight, bias, training, momentum, eps)

In [10]:
class OmniglotNet(nn.Module):
    '''
    Model as described in the reference paper,
    source: https://github.com/jakesnell/prototypical-networks/blob/f0c48808e496989d01db59f86d4449d7aee9ab0c/protonets/models/few_shot.py#L62-L84
    '''
    def __init__(self, x_dim=1, hid_dim=64, z_dim=64):
        super(OmniglotNet, self).__init__()
        self.encoder = nn.Sequential(OrderedDict([
            ('block1', conv_block(x_dim, hid_dim)),
            ('block2', conv_block(hid_dim, hid_dim)),
            ('block3', conv_block(hid_dim, hid_dim)),
            ('block4', conv_block(hid_dim, z_dim)),
        ]))

    def forward(self, x, weights=None):
        if weights is None:
            x = self.encoder(x)
        else:
            x = F.conv2d(x, weights['encoder.block1.conv.weight'], weights['encoder.block1.conv.bias'])
            x = batchnorm(x, weight=weights['encoder.block1.bn.weight'], bias=weights['encoder.block1.bn.bias'])
            x = F.relu(x)
            x = F.max_pool2d(x, 2, 2)
            x = F.conv2d(x, weights['encoder.block2.conv.weight'], weights['encoder.block2.conv.bias'])
            x = batchnorm(x, weight=weights['encoder.block2.bn.weight'], bias=weights['encoder.block2.bn.bias'])
            x = F.relu(x)
            x = F.max_pool2d(x, 2, 2)
            x = F.conv2d(x, weights['encoder.block3.conv.weight'], weights['encoder.block3.conv.bias'])
            x = batchnorm(x, weight=weights['encoder.block3.bn.weight'], bias=weights['encoder.block3.bn.bias'])
            x = F.relu(x)
            x = F.max_pool2d(x, 2, 2)
            x = F.conv2d(x, weights['encoder.block4.conv.weight'], weights['encoder.block4.conv.bias'])
            x = batchnorm(x, weight=weights['encoder.block4.bn.weight'], bias=weights['encoder.block4.bn.bias'])
            x = F.relu(x)
            x = F.max_pool2d(x, 2, 2)
        return x.view(x.size(0), -1)

In [11]:
class ResBlock(nn.Module):

    def __init__(self, in_channels, filters, pool_padding=0):
        super(ResBlock, self).__init__()
        self.conv1 = conv(in_channels, filters)
        self.bn1 = nn.BatchNorm2d(filters)
        self.relu1 = nn.LeakyReLU()
        self.conv2 = conv(filters, filters)
        self.bn2 = nn.BatchNorm2d(filters)
        self.relu2 = nn.LeakyReLU()
        self.conv3 = conv(filters, filters)
        self.bn3 = nn.BatchNorm2d(filters)
        self.relu3 = nn.LeakyReLU()
        self.conv4 = conv(in_channels, filters, kernel_size=1, padding=0)

        self.maxpool = nn.MaxPool2d(2, padding=pool_padding)
        self.dropout = nn.Dropout(p=0.9)

    def forward(self, x):
        residual = self.conv4(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)

        out = self.conv3(out)
        out = self.bn3(out)
        out = self.relu3(out)

        out += residual
        out = self.maxpool(out)
        out = self.dropout(out)

        return out

### `snail.py`

In [12]:
class SnailFewShot(nn.Module):
    def __init__(self, N, K):
        # N-way, K-shot
        super(SnailFewShot, self).__init__()
        self.encoder = OmniglotNet()
        num_channels = 64 + N
        num_filters = int(math.ceil(math.log(N * K + 1, 2)))
        self.attention1 = AttentionBlock(num_channels, 64, 32)
        num_channels += 32
        self.tc1 = TCBlock(num_channels, N * K + 1, 128)
        num_channels += num_filters * 128
        self.attention2 = AttentionBlock(num_channels, 256, 128)
        num_channels += 128
        self.tc2 = TCBlock(num_channels, N * K + 1, 128)
        num_channels += num_filters * 128
        self.attention3 = AttentionBlock(num_channels, 512, 256)
        num_channels += 256
        self.fc = nn.Linear(num_channels, N)
        self.N = N
        self.K = K

    def forward(self, input, labels):
        x = self.encoder(input)
        batch_size = int(labels.size()[0] / (self.N * self.K + 1))

        # TODO: Maybe move this zeroing to process_torchmeta_batch() at train.py
        last_idxs = [(i + 1) * (self.N * self.K + 1) - 1 for i in range(batch_size)]
#         labels[last_idxs] = torch.Tensor(np.zeros((batch_size, labels.size()[1]))).cuda()
        labels[last_idxs] = torch.Tensor(np.zeros((batch_size, labels.size()[1])))

        x = torch.cat((x, labels), 1)
        x = x.view((batch_size, self.N * self.K + 1, -1))
        x = self.attention1(x)
        x = self.tc1(x)
        x = self.attention2(x)
        x = self.tc2(x)
        x = self.attention3(x)
        x = self.fc(x)
        return x

## Helper Functions

In [13]:
def get_acc(last_model, last_targets):
    _, preds = last_model.max(1)
    acc = torch.eq(preds, last_targets).float().mean()
    return acc.item()

In [14]:
# TODO: Update docstring
def process_torchmeta_batch(batch, NUM_CLS, NUM_SAMPLES):
    """
    Process batch from torchmeta dataset for the SNAIL model

    Parameters
    ----------
    batch : dict
        A dictionary given by the torchmeta dataset
    options : SimpleNamespace
        A namespace with configuration details.

    Returns
    -------
    input_images : torch.tensor
        Input images to the SNAIL model of shape (batch_size*(N*K+1), img_channels, img_height,
        img_width) where N and K denote the same variables as in the N-way K-shot problem.
    input_onehot_labels : torch.tensor
        Input one-hot labels to the SNAIL model of shape (batch_size*(N*K+1), N) where N and K
        denote the same variables as in the N-way K-shot problem. The last label for each (N*K+1)
        is not used since it is the target label.
    target_labels : torch.tensor
        Test set labels to evaluate the SNAIL model by comparing with its outputs. Has shape
        (batch_size).

    """
    train_inputs, train_labels = batch["train"]
    test_inputs, test_labels = batch["test"]

    # Select one image from N images in the test set
    chosen_indices = torch.randint(test_inputs.shape[1], size=(test_inputs.shape[0],))
    chosen_test_inputs = test_inputs[torch.arange(test_inputs.shape[0]), chosen_indices, :, :, :].unsqueeze(1)
    chosen_test_labels = test_labels[torch.arange(test_labels.shape[0]), chosen_indices].unsqueeze(1)

    # Concatenate train and test set for SNAIL-style input images and labels
    input_images = torch.cat((train_inputs, chosen_test_inputs), dim=1).reshape((-1, *train_inputs.shape[2:]))
    input_labels = torch.cat((train_labels, chosen_test_labels), dim=1).reshape((-1, *train_labels.shape[2:]))

    # Convert labels to one-hot
    input_onehot_labels = F.one_hot(input_labels).float()

    # Separate out target labels
    target_labels = input_labels[::(NUM_CLS * NUM_SAMPLES + 1)].long()

    # Move to correct device
#     if options.cuda:
#         input_images, input_onehot_labels = input_images.cuda(), input_onehot_labels.cuda()
#         target_labels = target_labels.cuda()

    return input_images, input_onehot_labels, target_labels

## Hyperparameters

In [15]:
N_TRAIN_EPISODES = 100
N_VAL_EPISODES = 100
N_TEST_EPISODES = 100

In [16]:
OUTPUT_DIR = "default"

In [17]:
ADAM_LR = 1e-4

In [18]:
TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

In [19]:
NUM_CLS = 5
NUM_SAMPLES = 1

In [20]:
if torch.cuda.is_available() and not options.cuda:
    print("WARNING: You have a CUDA device, so you should probably run with --cuda")

## Loop

Main script for training SNAIL on Omniglot.

In [21]:
def train_one_episode(model, optimizer, batch, NUM_CLS, NUM_SAMPLES):
    input_images, input_onehot_labels, target_labels = process_torchmeta_batch(batch, NUM_CLS, NUM_SAMPLES)
    predicted_labels = model(input_images, input_onehot_labels)[:, -1, :]
    loss = F.cross_entropy(predicted_labels, target_labels)
    acc = get_acc(predicted_labels, target_labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item(), acc

In [22]:
def test_one_episode(model, optimizer, batch, NUM_CLS, NUM_SAMPLES):
    input_images, input_onehot_labels, target_labels = process_torchmeta_batch(batch, NUM_CLS, NUM_SAMPLES)
    predicted_labels = model(input_images, input_onehot_labels)[:, -1, :]
    loss = F.cross_entropy(predicted_labels, target_labels)
    acc = get_acc(predicted_labels, target_labels)

    return loss.item(), acc

In [23]:
def train(model, optimizer, train_dataloader, val_dataloader):
    if val_dataloader is None:
        best_state = None
    train_loss_per_ep, train_acc_per_ep = [], []
    val_loss_per_ep, val_acc_per_ep = [], []
    best_val_acc = 0

    best_model_path = os.path.join(OUTPUT_DIR, 'best_model.pth')
    last_model_path = os.path.join(OUTPUT_DIR, 'last_model.pth')

    # Training phase
    model.train()
    for i, batch in tqdm(enumerate(train_dataloader), total=N_TRAIN_EPISODES):
        if i >= N_TRAIN_EPISODES: break

        train_loss, train_acc = train_one_episode(model, optimizer, batch, NUM_CLS, NUM_SAMPLES)
        train_loss_per_ep.append(train_loss)
        train_acc_per_ep.append(train_acc)

    # TODO: Don't average all episodes here
    avg_train_loss = np.mean(train_loss)
    avg_train_acc = np.mean(train_acc)
    print('Avg Train Loss: {}, Avg Train Acc: {}'.format(avg_train_loss, avg_train_acc))

    # Validation phase
    model.eval()
    for i, batch in tqdm(enumerate(val_dataloader), total=N_VAL_EPISODES):
        if i >= N_VAL_EPISODES: break

        val_loss, val_acc = test_one_episode(model, optimizer, batch, NUM_CLS, NUM_SAMPLES)
        val_loss_per_ep.append(val_loss)
        val_acc_per_ep.append(val_acc)

    avg_val_loss = np.mean(val_loss)
    avg_val_acc = np.mean(val_acc)
    postfix = ' (Best)' if avg_val_acc >= best_val_acc else ' (Best: {})'.format(best_val_acc)
    print('Avg Val Loss: {}, Avg Val Acc: {}{}'.format(avg_val_loss, avg_val_acc, postfix))

    if avg_val_acc >= best_val_acc:
        torch.save(model.state_dict(), best_model_path)
        best_val_acc = avg_val_acc
        best_state = model.state_dict()

    for name in ['train_loss_per_ep', 'train_acc_per_ep', 'val_loss_per_ep', 'val_acc_per_ep']:
        with open(os.path.join(OUTPUT_DIR, name + '.txt'), 'w') as f:
            for item in locals()[name]:
                f.write("%s\n" % item)

    torch.save(model.state_dict(), last_model_path)

    return best_state, best_val_acc, train_loss_per_ep, train_acc_per_ep, val_loss_per_ep, val_acc_per_ep

In [24]:
def test(model, test_dataloader):
    """
    Test model on given dataset and options.
    """
    model.eval()
    test_loss_per_ep, test_acc_per_ep = [], []
    for i, batch in tqdm(enumerate(test_dataloader), total=N_TEST_EPISODES):
        if i >= N_TEST_EPISODES: break

        test_loss, test_acc = test_one_episode(model, optimizer, batch, NUM_CLS, NUM_SAMPLES)
        test_loss_per_ep.append(val_loss)
        test_acc_per_ep.append(val_acc)

    avg_test_loss = np.mean(test_loss_per_ep)
    avg_test_acc = np.mean(test_acc_per_ep)
    print('Avg Test Loss: {} Avg Test Acc: {}'.format(avg_test_loss, avg_test_acc))

    return avg_test_loss, avg_test_acc

In [25]:
# Setup dataset
train_dataset = omniglot("data", ways=5, shots=1, test_shots=1, meta_train=True, download=True)
train_dataloader = BatchMetaDataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=2)
val_dataset = omniglot("data", ways=5, shots=1, test_shots=1, meta_val=True, download=True)
val_dataloader = BatchMetaDataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, num_workers=2)
test_dataset = omniglot("data", ways=5, shots=1, test_shots=1, meta_test=True, download=True)
test_dataloader = BatchMetaDataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, num_workers=2)
# Setup model
model = SnailFewShot(NUM_CLS, NUM_SAMPLES)
# model = model.cuda() if options.cuda else model
# Setup optimizer
optimizer = optim.Adam(params=model.parameters(), lr=ADAM_LR)

In [26]:
# Train model
train_result = train(
    model=model,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
)
best_state, best_acc, train_loss, train_acc, val_loss, val_acc = train_result

100%|████████████████████████████████████████████| 100/100 [00:56<00:00,  1.77it/s]

Avg Train Loss: 1.5750852823257446, Avg Train Acc: 0.3125



  0%|                                                      | 0/100 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc77da82790>
Traceback (most recent call last):
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if w.is_alive():
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc77da82790>
Traceback (most recent call last):
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-bo

Avg Val Loss: 1.6460012197494507, Avg Val Acc: 0.15625 (Best)


In [27]:
# Test last model
test(
    model=model,
    test_dataloader=test_dataloader,
)

Testing with last model..


  0%|                                                      | 0/100 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc77da82790>
Traceback (most recent call last):
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
Exception ignored in:     self._shutdown_workers()<function _MultiProcessingDataLoaderIter.__del__ at 0x7fc77da82790>
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers

    if w.is_alive():Traceback (most recent call last):
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__

  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.get

Avg Test Loss: 1.620465074777603 Avg Test Acc: 0.1946875


(1.620465074777603, 0.1946875)

In [28]:
# Test best model
model.load_state_dict(best_state)
test(
    test_dataloader=test_dataloader,
    model=model,
)

Testing with best model..


  0%|                                                      | 0/100 [00:00<?, ?it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fc77da82790>
Traceback (most recent call last):
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1324, in __del__
    self._shutdown_workers()
  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    Exception ignored in: if w.is_alive():
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fc77da82790>  File "/home/seungjaeryanlee/anaconda3/envs/meta-learning-book/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process

Traceback (most recent call last):
Exception ignored in:   File "/home/seungjaeryanlee/anaconda3/

Avg Test Loss: 1.620465074777603 Avg Test Acc: 0.1946875


(1.620465074777603, 0.1946875)